In [1]:
!pip install gradio scikit-learn pandas

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.9/46.9 MB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.2/322.2 kB 14.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.4/11.4 MB 27.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.4/62.4 kB 1.4 MB/s eta 0:00:00


In [2]:
import gradio as gr
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split

In [3]:
# Load dataset
iris = load_iris()
X = pd.DataFrame(iris.data, columns=iris.feature_names)
y = iris.target

# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [4]:
# Create and train model
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Check accuracy
print(f"Model accuracy: {model.score(X_test, y_test):.2f}")

Model accuracy: 1.00


In [5]:
def predict_flower(sepal_length, sepal_width, petal_length, petal_width):
    # Create input array
    input_data = [[sepal_length, sepal_width, petal_length, petal_width]]

    # Make prediction
    prediction = model.predict(input_data)[0]
    probability = model.predict_proba(input_data)[0]

    # Return results
    return {
        "Predicted Class": iris.target_names[prediction],
        "Class Probabilities": {
            iris.target_names[0]: float(probability[0]),
            iris.target_names[1]: float(probability[1]),
            iris.target_names[2]: float(probability[2])
        }
    }

In [6]:
# Define input components
inputs = [
    gr.Slider(4.0, 8.0, step=0.1, label="Sepal Length (cm)"),
    gr.Slider(2.0, 4.5, step=0.1, label="Sepal Width (cm)"),
    gr.Slider(1.0, 7.0, step=0.1, label="Petal Length (cm)"),
    gr.Slider(0.1, 2.5, step=0.1, label="Petal Width (cm)")
]

# Create interface
interface = gr.Interface(
    fn=predict_flower,
    inputs=inputs,
    outputs=[
        gr.Label(label="Predicted Class"),
        gr.Label(label="Class Probabilities")
    ],
    title="Iris Flower Classifier",
    description="Predict iris flower species based on measurements using Random Forest"
)

In [7]:
interface.launch(share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://3d520c77e3b8cb1222.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [8]:
def predict_flower(sepal_length, sepal_width, petal_length, petal_width):
    try:
        # Convert to float and validate
        values = [float(v) for v in [sepal_length, sepal_width, petal_length, petal_width]]
        input_data = [values]

        # Predict
        prediction = model.predict(input_data)[0]
        proba = model.predict_proba(input_data)[0]

        return {
            "Predicted Class": iris.target_names[prediction],
            "Probabilities": dict(zip(iris.target_names, [float(p) for p in proba]))
        }
    except Exception as e:
        return {"Error": f"Invalid input: {str(e)}"}

In [9]:
# Step 1: Install required libraries
!pip install gradio scikit-learn pandas numpy

# Step 2: Import libraries
import gradio as gr
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Step 3: Load and prepare data
iris = load_iris()
X = pd.DataFrame(iris.data, columns=iris.feature_names)
y = iris.target

# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Scale data (important for real-world datasets)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Step 4: Train model
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)
print(f"Model accuracy: {model.score(X_test, y_test):.2f}")

# Step 5: Prediction function with error handling
def predict_flower(sepal_length, sepal_width, petal_length, petal_width):
    try:
        # Validate inputs
        inputs = [sepal_length, sepal_width, petal_length, petal_width]
        if any(v is None for v in inputs):
            raise ValueError("All fields must be filled")

        # Convert to numpy array and scale
        input_array = np.array([inputs], dtype=np.float32)
        scaled_input = scaler.transform(input_array)

        # Predict
        prediction = model.predict(scaled_input)[0]
        probabilities = model.predict_proba(scaled_input)[0]

        return {
            "Predicted Class": iris.target_names[prediction],
            "Confidence (%)": {
                iris.target_names[0]: round(probabilities[0]*100, 2),
                iris.target_names[1]: round(probabilities[1]*100, 2),
                iris.target_names[2]: round(probabilities[2]*100, 2)
            }
        }
    except Exception as e:
        return {"Error": str(e)}

# Step 6: Create interface with default values
with gr.Blocks() as demo:
    gr.Markdown("## Iris Flower Classifier 🌸")
    with gr.Row():
        with gr.Column():
            sl = gr.Slider(4.0, 8.0, value=5.8, step=0.1, label="Sepal Length (cm)")
            sw = gr.Slider(2.0, 4.5, value=3.0, step=0.1, label="Sepal Width (cm)")
            pl = gr.Slider(1.0, 7.0, value=4.0, step=0.1, label="Petal Length (cm)")
            pw = gr.Slider(0.1, 2.5, value=1.2, step=0.1, label="Petal Width (cm)")
            submit_btn = gr.Button("Predict")
        with gr.Column():
            label_output = gr.Label(label="Prediction Result")
            json_output = gr.JSON(label="Class Probabilities")

    submit_btn.click(
        fn=predict_flower,
        inputs=[sl, sw, pl, pw],
        outputs=[label_output, json_output]
    )

# Step 7: Launch
demo.launch(share=True)

Model accuracy: 1.00
Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://3bccd1775329ede686.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
